# Combining the .h5 files for the first time

from torchani.datasets.utils import concatenate
concat_path = path / 'ALL_COMBINED.h5'
print(concat_path)
ds = concatenate(ds, concat_path, delete_originals=False)


In [17]:
import torch
import torchani
from torchani.datasets import ANIDataset
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from torchani.units import hartree2kcalmol
from tqdm import tqdm

In [20]:
path = Path('/home/nick/Datasets/comp6v2/ALL_COMBINED.h5')

In [21]:
ds = ANIDataset(locations=path, names='comp6v2')
print('Conformer groups in dataset', len(ds.keys()))
print(ds.grouping)

Conformers in dataset 3376
by_formula


ds.regroup_by_formula()

In [22]:
device = torch.device('cpu')
ani2x = torchani.models.ANI2x()

/home/nick/TorchANI_sandbox/torchani/models.py:103: UserWarning: The default is now to accept atomic numbers as indexes, do not set periodic_table_index=True. if you need to accept raw indices set periodic_table_index=False
  warnings.warn("The default is now to accept atomic numbers as indexes,"


In [30]:
print(ds)

Name: comp6v2
ANI h5py store:
{'Conformer groups': 3376,
 'Conformers': '157,728',
 'Properties': ['atomic_charges_cm5',
                'atomic_charges_hirshfeld',
                'atomic_dipoles_hirshfeld',
                'coordinates',
                'energies',
                'forces',
                'species'],
 'Store Metadata': {}}


In [24]:
ds.properties

{'atomic_charges_cm5',
 'atomic_charges_hirshfeld',
 'atomic_dipoles_hirshfeld',
 'coordinates',
 'energies',
 'forces',
 'species'}

In [27]:
count = 0

df = pd.DataFrame(columns=['Formula','Species','Avg_AE','Stdev','QBC'])
count = 0
formula_list = []
species_list = []
avg_ae_list = []
stdev_list = []
qbc_list = []

for index, conformer in tqdm(enumerate(ds.iter_conformers())):
    species = conformer['species'][None]
    coordinates = conformer['coordinates'][None]
    assert len(species) == len(coordinates)
    ani_input = (species,coordinates)    
    _, energies, qbc_factor = ani2x.energies_qbcs(ani_input)
    species_index, ae = ani2x.atomic_energies(ani_input, average=False, with_SAEs=False)
    avg_ae = ae.mean(0).tolist()
    stdev_ae = ae.std(0).tolist()
    coef_var = (ae.std(0)/abs(ae.mean(0))).tolist()
    
    for atoms in avg_ae[0]:
        formula_list.append(formula)
        # Want a chemical formula and qbc factor attached to each atom type
        qbc_list.append(qbc_factor.detach().numpy())
    
    species_list.extend(species.tolist()[0])
    avg_ae_list.extend(avg_ae[0])
    stdev_list.extend(stdev_ae[0])
    
    break
    '''
    count += 1
    print('iter #',count)
    if count == 100:
        break
    '''

df['Species'] = species_list
df['Avg_AE'] = avg_ae_list
df['Stdev'] = stdev_list
df['Formula'] = formula_list
df['QBC'] = qbc_list
print(df)
df.to_csv('df_comp6v2.csv')

0it [00:00, ?it/s]


                                               Formula  Species    Avg_AE  \
0    {'energies': tensor(-7654.5016, dtype=torch.fl...        6 -0.004423   
1    {'energies': tensor(-7654.5016, dtype=torch.fl...        6 -0.027310   
2    {'energies': tensor(-7654.5016, dtype=torch.fl...        8 -0.011999   
3    {'energies': tensor(-7654.5016, dtype=torch.fl...        1 -0.004328   
4    {'energies': tensor(-7654.5016, dtype=torch.fl...        1 -0.002906   
..                                                 ...      ...       ...   
307  {'energies': tensor(-7654.5016, dtype=torch.fl...        6 -0.004897   
308  {'energies': tensor(-7654.5016, dtype=torch.fl...        1 -0.011200   
309  {'energies': tensor(-7654.5016, dtype=torch.fl...        1 -0.003887   
310  {'energies': tensor(-7654.5016, dtype=torch.fl...        1 -0.003876   
311  {'energies': tensor(-7654.5016, dtype=torch.fl...        1 -0.001940   

        Stdev              QBC  
0    0.009111  [0.00078292226]  
1    0.00